In [37]:
# Build a web application that scrapes various websites for data related to the Mission to Mars and displays the information in a single HTML page.
# Initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter

In [38]:
# Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import pandas as pd

In [39]:
# Mac Users
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html

!which chromedriver

/usr/local/bin/chromedriver


In [40]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## NASA Mars News

In [4]:
# Collect latest News Title and Paragraph Text
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [41]:
# Create variables, Beautiful Soup object
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
# Examine the results, then determine element that contains sought info results are returned as an iterable list
results = soup.find('ul', class_='item_list')

In [7]:
listitem = results.find('li', class_='slide')

In [8]:
title = listitem.find('div', class_='content_title').get_text()

In [9]:
body = listitem.find('div', class_='article_teaser_body').get_text()

In [10]:
title

"NASA's Perseverance Rover Mission Getting in Shape for Launch"

In [11]:
body

'Stacking spacecraft components on top of each other is one of the final assembly steps before a mission launches to the Red Planet. '

## JPL Mars Space Images - Featured Image

In [20]:
# Visit the url for JPL Featured Space Image
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [21]:
# Use splinter to navigate to the site and find the image url for the current Featured Mars Image
browser.find_by_id("full_image").click()

In [22]:
browser.is_element_present_by_text('more info', wait_time=1)
browser.links.find_by_partial_text('more info').click()

In [23]:
# Create variables, Beautiful Soup object
html = browser.html
soup = bs(html, 'html.parser')

In [24]:
# Assign the url string to a variable called featured_image_url
image = soup.select_one('figure.lede a img').get("src")

# Make sure to find the image url to the full size .jpg image.
# Make sure to save a complete url string for this image.
image_url = f"https://jpl.nasa.gov{image}"

In [25]:
image_url

'https://jpl.nasa.gov/spaceimages/images/largesize/PIA19330_hires.jpg'

## Mars Weather

In [28]:
# Visit Mars Weather Twitter url
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
time.sleep(5)

In [42]:
# Create variables, Beautiful Soup object
html = browser.html
soup = bs(html, 'html.parser')

In [30]:
# Note: this includes the Twitter account name and handle
# Examine the results, then determine element that contains sought info results are returned as an iterable list
# Save tweet text for weather report as a variable called 
mars_weather = soup.find('article', class_='css-1dbjc4n').get_text()
mars_weather

'Mars Weather@MarsWxReport·May 16InSight sol 522 (2020-05-16) low -93.2ºC (-135.7ºF) high -1.7ºC (29.0ºF)\nwinds from the SW at 5.1 m/s (11.3 mph) gusting to 18.8 m/s (42.1 mph)\npressure at 7.00 hPa619'

## Mars Facts

In [31]:
# Visit the Mars Facts webpage. Scrape table containing facts about planet (e.g. diameter, mass, etc.)

# Pass in URL
df = pd.read_html('https://space-facts.com/mars/')[0]

# Pull table on page as dataframe. If list, select the index for table wanted
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [32]:
# Convert data to HTML table string
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

## Mars Hemispheres

In [43]:
# Visit url
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [45]:
# Create dictionary
hemisphere_image_urls = []

# Get list of hemispheres
links = browser.find_by_css("a.product-item h3")

# Loop through links, click link, find sample anchor, return href
for i in range(len(links)):
    hemisphere = {}
    
    # Find elements on each loop to avoid stale element exceptions
    browser.find_by_css("a.product-item h3")[i].click()
    
    # Find sample image anchor tag, extract href
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Navigate back
    browser.back()

In [46]:
hemisphere_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]